# Fibre packer -- first draft

*Author: Vedrana Andersen Dahl (vand@dtu.dk)*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vedranaa/fibre-pack/blob/main/fibre_packer_demo.ipynb)



In [1]:
import torch
import plotly.express as px
import plotly.graph_objects as go
from tqdm.notebook import tqdm
import os
if not os.path.isfile('fibre_packer.py'):
    !wget 'https://raw.githubusercontent.com/vedranaa/fibre-pack/main/fibre_packer.py' -q
import fibre_packer as fp

In [2]:
def optimize_slice_points(p, radii, c, iters=2000):
    delta = 0.01 * radii.mean()
    n = 3
    N = len(radii)
    min_d = fp.minimal_distance(radii)
    p.requires_grad = True
    p.to(device)
    optimizer = torch.optim.Adam([p], lr=0.1)
    loss_contributions = []
    progress_bar = tqdm(range(iters), bar_format='{l_bar}{bar}|{n_fmt}/{total_fmt}')
    for iter in progress_bar:  
        optimizer.zero_grad()   
        d = fp.pairwise_distance(p)
        overlap = fp.overlap_penalty(d, min_d, delta)
        protrusion = fp.protrusion_penalty(p, radii, c)
        separation = fp.separation_penalty(d, radii, n, delta)
        loss = overlap + N * protrusion + 1/N * separation
        loss.backward()
        optimizer.step()
        loss_contributions.append((overlap.item(), N * protrusion.item(), 1/N * separation.item()))
        progress_bar.set_description(f"Overlap {overlap:.2f}, " + 
            f"Protrusion {protrusion:.2f}", refresh=True)
    
    p = p.detach()
    overlap = fp.overlap_penalty(d, min_d)
    protrusion = fp.protrusion_penalty(p, radii, c)
    loss_contributions = {k:list(v) for k, v in 
            zip(['overlap', 'protrusion', 'separation'], zip(*loss_contributions))}
    return p, (overlap, protrusion), loss_contributions


In [ ]:
c = 40  # Domain radius
r0 = 2  # Mean fibre radius
fvf = 70  # Desired fibre volume fraction
Z = 20 # Number of slices

device = fp.select_device()

radii = fp.initialize_radii(c, fvf, r0, 0.1 * r0)
N = len(radii)
fp.show_radii_distribution(radii)


In [ ]:
p0 = fp.initialize_slice_points(c - r0, N)
fp.show_slice(p0, radii, c, title='First slice (p0) initial')

p0, (overlap, protrusion), losses = optimize_slice_points(p0, radii, c)
fp.show_slice(p0, radii, c, 
    title=f'First slice (p0) optimized. Overlap {overlap:.2f}, protrusion {protrusion:.2f}')
fp.show_losses(losses)

In [ ]:
pZ = p0.clone()
pZ = fp.rotate_bundle(pZ, radii, (c/2, 0), c/2.5, -torch.pi/2)
pZ = fp.rotate_bundle(pZ, radii, (-c/2, 0), c/2, -torch.pi/3)
pZ = fp.rotate_bundle(pZ, radii, (0, 0), c, torch.pi/4)
pz = fp.swap_points(pZ)

fp.show_slice(pZ, radii, c, title='Last slice (pZ) initial')
pZ, (overlap, protrusion), losses = optimize_slice_points(pZ, radii, c)
fp.show_slice(pZ, radii, c, 
    title=f'Last slice (pZ) optimized. Overlap {overlap:.2f}, protrusion {protrusion:.2f}')
fp.show_losses(losses)


In [ ]:
configuration = fp.interpolate_configuration(p0, pZ, Z)
fp.show_3D_configuration(configuration, title='Initial configuration')
fp.animate_configuration(configuration, title='Initial configuration')

In [ ]:
min_d = fp.minimal_distance(radii)
delta = 0.01 * radii.mean()
configuration.requires_grad = True
configuration.to(device)
optimizer = torch.optim.Adam([configuration], lr=0.1)
loss_contributions = []
iters = 2000
progress_bar = tqdm(range(iters), bar_format='{l_bar}{bar}|{n_fmt}/{total_fmt}')
for iter in progress_bar:  
    optimizer.zero_grad()   
    d = fp.pairwise_distance(configuration)
    overlap = fp.overlap_penalty(d, min_d, delta)
    protrusion = fp.protrusion_penalty(configuration, radii, c)
    stretching = fp.stretching_penalty(configuration)
    bending = fp.bending_penalty(configuration)
    boundary = fp.boundary_penalty(configuration, p0, pZ)
    loss = overlap + N * protrusion + 1/N * stretching + 2/N * bending + N * boundary
    loss.backward()
    optimizer.step()
    loss_contributions.append((overlap.item(), N * protrusion.item(), 
        1/N * stretching.item(), 2/N * bending.item(), N * boundary.item()))
    progress_bar.set_description(f"Over. {overlap.item():.2f}, " + 
            f"Prot. {protrusion.item():.1f}, " +
            f"Str. {stretching.item():.1f}, " +
            f"Bend. {bending.item():.1f}, " +
            f"Boun. {boundary.item():.1f}",
            refresh=True)

loss_contributions = {k:list(v) for k, v in 
        zip(['overlap', 'protrusion', 'stretching', 'bending', 'boundary'], zip(*loss_contributions))}
fp.show_losses(loss_contributions)

In [ ]:
fp.show_3D_configuration(configuration, title='Optimized configuration')
fp.animate_slices(configuration, radii, c, title='Optimized configuration')